In [2]:
import pandas as pd

import plotly.graph_objects as go
from IPython.display import SVG, display
import plotly.io as pio 

from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings("ignore")

# Análise dos dados - IMDB Jogos

## Dicionário de dados

| Índice | Nome da Coluna | Tipo de Coluna | Descrição
|:--: | :--: | :--: | :--: |
|0  | title                    | categorica nominal | Title
|1  | console                  | categorica nominal | A consola em que o jogo foi lançado
|2  | alcohol_reference        | categorica nominal | Referência a bebidas alcoólicas e/ou imagens de bebidas alcoólicas. 
|3  | animated_blood           | categorica nominal | Representações descoloradas e/ou irrealistas de sangue.
|4  | blood                    | categorica nominal | Representações de sangue.
|5  | blood_and_gore           | categorica nominal | Representações de sangue ou de mutilação de partes do corpo. 
|6  | cartoon_violence         | categorica nominal | Acções violentas que envolvem situações e personagens de desenhos animados.
|7  | crude_humor              | categorica nominal | Representações ou diálogos que envolvam cenas vulgares, incluindo humor de "casa de banho".
|8  | drug_reference           | categorica nominal | Referência e/ou imagens de drogas ilegais.
|9  | fantasy_violence         | categorica nominal | Acções violentas de natureza fantasiosa, envolvendo personagens humanas ou não humanas em situações facilmente distinguíveis da vida real.
|10 | intense_violence         | categorica nominal | Representações gráficas e realistas de conflitos físicos. Envolve sangue extremo e/ou realista, armas e representações de ferimentos e mortes humanas. 
|11 | language                 | categorica nominal | Utilização moderada de palavrões.
|12 | lyrics                   | categorica nominal | Referências a profanação, sexualidade, violência, álcool ou consumo de drogas na música.
|13 | mature_humor             | categorica nominal | Representações ou diálogos que envolvam humor "adulto", incluindo referências sexuais.
|14 | mild_blood               | categorica nominal | Algum sangue
|15 | mild_cartoon_violence    | categorica nominal | Algumas acções violentas envolvendo desenhos animados.
|16 | mild_fantasy_violence    | categorica nominal | Algumas acções violentas de natureza fantasiosa.
|17 | mild_language            | categorica nominal | Utilização ligeira a moderada de palavrões.
|18 | mild_lyrics              | categorica nominal | Referências ligeiras a profanação, sexualidade, violência, álcool ou consumo de drogas na música.
|19 | mild_suggestive_themes   | categorica nominal | algumas referências ou materiais provocadores
|20 | mild_violence            | categorica nominal | Algumas cenas que envolvem conflitos agressivos.
|21 | no_descriptors           | categorica nominal | Não há descritores de conteúdo. 
|22 | nudity                   | categorica nominal | Representações gráficas ou prolongadas de nudez.
|23 | partial_nudity           | categorica nominal | Representações breves e/ou ligeiras de nudez.
|24 | sexual_content           | categorica nominal | Representações não explícitas de comportamentos sexuais, incluindo eventualmente nudez parcial.
|25 | sexual_themes            | categorica nominal | Referências a sexo ou sexualidade. 
|26 | simulated_gambling       | categorica nominal | O jogador pode jogar sem apostar ou apostar dinheiro ou moeda real. 
|27 | strong_janguage          | categorica nominal | Utilização explícita e/ou frequente de palavrões.
|28 | strong_sexual_content    | categorica nominal | Representações explícitas e/ou frequentes de comportamentos sexuais, possivelmente incluindo nudez.
|29 | suggestive_themes        | categorica nominal | Referências ou materiais provocadores.
|30 | use_of_alcohol           | categorica nominal | O consumo de bebidas alcoólicas.
|31 | use_of_drugs_and_alcohol | categorica nominal | O consumo de bebidas alcoólicas e de drogas.
|32 | violence                 | categorica nominal | Cenas de conflito agressivo. Pode conter desmembramentos sem sangue.
|33 | esrb_rating              | categorica nominal | Classificação: RP - EC - E - E+10 - T - M - A

## Etapas de desenvolvimento

<ul>
<li>Análise de dimensionalidade dos dados</li>
<li>Análise de nulos e tipagem das variáveis</li>
<li>Análise visual de dimensionalidade da coluna alvo</li>
<li>Análise empírica de seleção de colunas</li>
<li>Seleção de colunas utilizando RFECV e RFE</li>
</ul>

# Análise de dados

## Análise de dimensionalidade de dados

In [3]:
df = pd.read_csv(r'.\Video_games_esrb_rating.csv')

In [4]:
print(f'A base de dados contêm: {df.shape[0]} linhas')
print(f'A base de dados contêm: {df.shape[1]} colunas')

A base de dados contêm: 1895 linhas
A base de dados contêm: 34 colunas


## Análise de nulos e tipagem das variáveis

In [5]:
def search_nulls(dataframe):
    if df[df.isnull()].dropna().shape[0] == 0:
        print('A base de dados não contêm nulos !')
    else:
        print(f'A base de dados contêm: {df[df.isnull()].dropna().shape[1]} nulos')

In [6]:
search_nulls(df)

A base de dados não contêm nulos !


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1895 entries, 0 to 1894
Data columns (total 34 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   title                     1895 non-null   object
 1   console                   1895 non-null   int64 
 2   alcohol_reference         1895 non-null   int64 
 3   animated_blood            1895 non-null   int64 
 4   blood                     1895 non-null   int64 
 5   blood_and_gore            1895 non-null   int64 
 6   cartoon_violence          1895 non-null   int64 
 7   crude_humor               1895 non-null   int64 
 8   drug_reference            1895 non-null   int64 
 9   fantasy_violence          1895 non-null   int64 
 10  intense_violence          1895 non-null   int64 
 11  language                  1895 non-null   int64 
 12  lyrics                    1895 non-null   int64 
 13  mature_humor              1895 non-null   int64 
 14  mild_blood              

## Análise visual de dimensionalidade da coluna alvo

In [8]:
fig = go.Figure()
color = ['blue', 'red','yellow', 'purple']
fig.add_trace(
    go.Bar(
        x = df['esrb_rating'].value_counts().index,
        y = df['esrb_rating'].value_counts().values,
        marker_color = color
    )
)
fig.update_layout(
    title = 'Categorias da base de dados'
    ,width = 500, height = 500)

fig.show()

Através do gráfico acima é possível verificar que a base está desbalanceada, então é possível considerar que será necessários efetuar alguma estrategia de balanceamento de dados (undersampling ou oversampling)

In [9]:
rename_esrb = {'E': 0,'ET': 1,'M': 2,'T': 3}
df_corr = df.copy()
df_corr.replace(rename_esrb, inplace=True)
df_corr.drop('title', axis = 1, inplace=True) 
df_corr = df_corr.corr(method='spearman')

In [10]:
rename_esrb = {'E': 0,'ET': 1,'M': 2,'T': 3}
df_corr = df.copy()
df_corr.replace(rename_esrb, inplace=True)

## Análise empírica de seleção de colunas

In [11]:
fig = go.Figure()
fig.add_trace(
    go.Heatmap(
        x=df_corr.columns,
        y=df_corr.index,
        z= df_corr,
        text=df_corr.values,
        texttemplate='%{text:.2f}'
        )
)
fig.update_layout(
    width = 1200, height = 1200,
    title_text = f'Compotamento da variável {df.columns[1]}',
    barmode='relative',
    )
fig.show()

In [12]:
rename_esrb = {'E': 0,'ET': 1,'M': 2,'T': 3}
df_corr = df.copy()
df_corr.replace(rename_esrb, inplace=True)
df_corr.drop('title', axis = 1, inplace=True) 
results_corr = {}
for i in ['spearman', 'kendall', 'pearson']:
    df_corr = df_corr.corr(method='spearman')
    values = pd.concat([df_corr[df_corr['esrb_rating']>0.1]['esrb_rating'],df_corr[df_corr['esrb_rating']<-0.1]['esrb_rating']]) # Foi admitido 0.1 tanto positivo quanto negativo
    results_corr[i] = values

In [13]:
results_corr

{'spearman': blood                     0.410961
 blood_and_gore            0.131583
 crude_humor               0.139683
 intense_violence          0.124627
 language                  0.196516
 mild_blood                0.131299
 mild_suggestive_themes    0.110542
 sexual_content            0.103436
 sexual_themes             0.109425
 simulated_gambling        0.179591
 suggestive_themes         0.269292
 violence                  0.237895
 esrb_rating               1.000000
 mild_cartoon_violence    -0.111653
 mild_fantasy_violence    -0.280977
 no_descriptors           -0.478738
 Name: esrb_rating, dtype: float64,
 'kendall': blood                       0.548797
 blood_and_gore              0.352273
 drug_reference              0.364502
 fantasy_violence            0.154926
 intense_violence            0.197861
 language                    0.428810
 lyrics                      0.471925
 mature_humor                0.280271
 mild_blood                  0.184158
 mild_suggestive_themes

## Seleção de colunas utilizando RFECV e RFE

### RFECV - Recursive feature elimination com Cross Validation

In [14]:
# Para a entrada dos dados será efetuada uma técnica chamada de dummies
x = pd.get_dummies(df.drop(['title', 'esrb_rating'], axis = 1).astype(str))
y = df['esrb_rating']
rfe_selector = RFECV(estimator = LogisticRegression(random_state=10)).fit(x, y)

In [15]:
for i in range(rfe_selector.get_feature_names_out().shape[0]):
    print(f'Coluna selecionada: {rfe_selector.get_feature_names_out()[i]}\nSelecionada {rfe_selector.support_[i]}\nRank: {rfe_selector.ranking_[i]}\n')

Coluna selecionada: alcohol_reference_0
Selecionada False
Rank: 9

Coluna selecionada: alcohol_reference_1
Selecionada False
Rank: 10

Coluna selecionada: animated_blood_1
Selecionada True
Rank: 1

Coluna selecionada: blood_0
Selecionada True
Rank: 1

Coluna selecionada: blood_1
Selecionada False
Rank: 5

Coluna selecionada: blood_and_gore_0
Selecionada True
Rank: 1

Coluna selecionada: blood_and_gore_1
Selecionada True
Rank: 1

Coluna selecionada: cartoon_violence_0
Selecionada True
Rank: 1

Coluna selecionada: cartoon_violence_1
Selecionada True
Rank: 1

Coluna selecionada: crude_humor_0
Selecionada True
Rank: 1

Coluna selecionada: crude_humor_1
Selecionada True
Rank: 1

Coluna selecionada: drug_reference_0
Selecionada True
Rank: 1

Coluna selecionada: drug_reference_1
Selecionada True
Rank: 1

Coluna selecionada: fantasy_violence_0
Selecionada True
Rank: 1

Coluna selecionada: fantasy_violence_1
Selecionada True
Rank: 1

Coluna selecionada: intense_violence_0
Selecionada True
Rank:

### RFE - Recursive feature elimination

In [26]:
x = pd.get_dummies(df.drop(['title', 'esrb_rating'], axis = 1).astype(str))
y = df['esrb_rating']
rfe_selector = RFECV(estimator = LogisticRegression(random_state=10), n_features_to_select=16).fit(x, y)

TypeError: RFECV.__init__() got an unexpected keyword argument 'n_features_to_select'

In [ ]:
for i in range(rfe_selector.get_feature_names_out().shape[0]):
    print(f'Coluna selecionada: {rfe_selector.get_feature_names_out()[i]}\nSelecionada {rfe_selector.support_[i]}\nRank: {rfe_selector.ranking_[i]}\n')

Coluna selecionada: alcohol_reference_0
Selecionada False
Rank: 9

Coluna selecionada: alcohol_reference_1
Selecionada False
Rank: 10

Coluna selecionada: animated_blood_1
Selecionada True
Rank: 1

Coluna selecionada: blood_0
Selecionada True
Rank: 1

Coluna selecionada: blood_1
Selecionada False
Rank: 5

Coluna selecionada: blood_and_gore_0
Selecionada True
Rank: 1

Coluna selecionada: blood_and_gore_1
Selecionada True
Rank: 1

Coluna selecionada: cartoon_violence_0
Selecionada True
Rank: 1

Coluna selecionada: cartoon_violence_1
Selecionada True
Rank: 1

Coluna selecionada: crude_humor_0
Selecionada True
Rank: 1

Coluna selecionada: crude_humor_1
Selecionada True
Rank: 1

Coluna selecionada: drug_reference_0
Selecionada True
Rank: 1

Coluna selecionada: drug_reference_1
Selecionada True
Rank: 1

Coluna selecionada: fantasy_violence_0
Selecionada True
Rank: 1

Coluna selecionada: fantasy_violence_1
Selecionada True
Rank: 1

Coluna selecionada: intense_violence_0
Selecionada True
Rank: